Question 1

In this assignment you will implement one or more algorithms for the traveling salesman problem, such as the dynamic programming algorithm covered in the video lectures.  Here is a data file describing a TSP instance: tsp.txt

The first line indicates the number of cities.  Each city is a point in the plane, and each subsequent line indicates the x- and y-coordinates of a single city.  

The distance between two cities is defined as the Euclidean distance --- that is, two cities at locations (x,y) and (z,w) have distance: squre root of (x - z)^2 + (y - w)^2 between them.  

In the box below, type in the minimum cost of a traveling salesman tour for this instance, rounded down to the nearest integer.

HINT: You might experiment with ways to reduce the data set size.  For example, trying plotting the points.  Can you infer any structure of the optimal solution?  Can you use that structure to speed up your algorithm?

In [1]:
import itertools as it
import math
import numpy as np
import time

def load(filename):
    """
        To load the data file into a list, having first element as count of the number of cities, and others as a list
        of the x- and y-coordinates of a single city
    """
    data = []
    with open(filename) as file:
        f = file.readlines()
        data.append(int(f[0].strip()))
        
        for lines in f[1:]:
            x, y = lines.strip().rsplit(" ")
            data.append([float(x), float(y)])
    return data
    


def generate_subsets(n):
    """
        To generate subsets of n items, that including starting point 1 and more than 1 element in the suset
        Returns the list subsets in the form of bitmask, ex. point 1 represent as 2 (since 2^1);
        and a hash map to map the bitmask to the list of points and its index in subsets
        ex: data is: [4, [0.0, 0.0], [4.0, 3.0], [4.0, 0.0], [0.0, 3.0]]
        subset is [2, 6, 10, 18, 14, 22, 26, 30]
        hashmap is {2: [[1],0], 6: [[1, 2], 1], 10: [[1, 3], 2] , 18: [[1, 4], 3], 14: [[1, 2, 3], 4], 
                        22: [[1, 2, 4], 5], 26: [[1, 3, 4], 6], 30: [[1, 2, 3, 4], 7]}
    """
    subsets = []
    hashmap = {}
    
    # find the subset of {2,..,n}, then add 1 to each subset to ensure each subset has 1
    # then calculate the bitmask and log to hashmap as {bitmask: subset list}
    j = 0
    for i in range(n+1):
        for items in it.combinations(range(2, n+1), i):
            subset = [1] + list(items) 
            bit_mask = 0
            
            for item in subset:
                bit_mask += (1<<item)    
            subsets.append(bit_mask)
            hashmap[bit_mask] = [subset, j]
            j += 1
    
    return subsets, hashmap
            

def cal_dist(data):
    """
        Given data list and compute the distance of every two points
    """

    dist_val = np.zeros([data[0]+1,data[0]+1])
    for i in range(1, data[0]):
        for k in range(i + 1, data[0] + 1):
            dist_val[i,k] = dist_val[k,i] = np.sqrt((data[k][0]-data[i][0])**2 + (data[k][1]-data[i][1])**2)
    return dist_val


def TSP_algorithm(filename):
    """
        Traveling Salesman Problem Algorithm using NP-Complete and Dynamic Programming
        Ideas are:
        1. Find out subsets S includes {1, 2, ..., n} that contain 1 using generate_subsets(n) above. The subsets are also 
            instance of subproblems.
        2. Loop through the subsets in ascending order of count of subset elements, in each subset S, for each point
            other than point 1 as an end point, find out the minimum distance from 1 to that point, using dynamic programming: 
            find out the minimum distance among all instances of A[(Set S-{j}, k)]+ c(kj), where (Set S-{j}, k) 
            reprsents subset S excluding point j, and having k, which is every point other than j in subset as end point, 
            and c(kj) represents distance of k and j
        3. Once finish above steps, should have different results of A[(Set{1,2,...n}, j)], where j is every point other 1
            in range 1 to n. 
        4. Last calculation is to find out the minimum value of A[(Set{1,2,...n}, j)] + c(j1), since last hop is from j
            back to 1, and since minimum value can come from any point j other than 1, it needs to loop through all points
            other than 1 again for j
    """   
    start = time.time()
    data = load(filename)
    end = time.time()
    print(f"Time spent on loading data is {end-start} second(s)")
    
    start = time.time()
    subsets, hashmap = generate_subsets(data[0])
    end = time.time()
    print(f"Time spent on generating subsets is {end-start} second(s)")
    
    start = time.time()
    distances = cal_dist(data)
    end = time.time()
    print(f"Time spent on calculating distance is {end-start} second(s)")
    
    start = time.time()
    
    result = np.ones((len(subsets),data[0]+1)) * np.inf
    
    # base cases of A[(S, 1)]: 1. result is 0 if the subset only has {1}; 2. all others are infinte to avoid visiting 
    # vertex twice since result[(S has all vertex, 1)] is the last instance we visit
    result[0,1] = 0
    
    # loop through subsets, since it's already in order when generating
    for bit in subsets:
        
        subset, index = hashmap[bit][0], hashmap[bit][1]
        
        if len(subset) == 1:
            continue
            
        elif len(subset) == 2:
            result[index, subset[1]] = distances[1, subset[1]]
            
        else:
            # loop throught nodes other than 1 in the subset, use it as the end point j 
            for node in subset:
                min_dist = math.inf

                if node != 1:
                    # pick a mid point from the subset other than node j, find the mimimum distance of A[S - {j}, k] + c(jk)
                    # among each mid point, where S is the subset, j is the last node and k is the mid point
                    # even k is called mid point here, it actually represents the node before j that get visited
                    new_bitmask = hashmap[bit-2**node][1]
                    for mid_point in subset:
                        if mid_point != node:
                            dist = result[new_bitmask, mid_point] + distances[mid_point, node]
                            if dist < min_dist:
                                min_dist = dist

                result[index, node] = min_dist
    
    min_dist = math.inf
    # subsets[-1] is the subset that has every element. loop through it to calculate the distance of {A[(1, 2,..., n}, j] 
    # + c(j1) and the minimum value of it is the minimum distance of TSP problem
    for node in hashmap[subsets[-1]][0]:
        if node != 1:
            dist = result[len(subsets)-1, node] + distances[1, node]
            if dist < min_dist:
                min_dist = dist
                
    end = time.time()
    print(f"Time spent on doing dynamic programming is {end-start} second(s)")
    print()
                
    return round(min_dist,2)
            
    
    
if __name__ == "__main__":
    minimum_distance = TSP_algorithm("tsp.txt")
    print(f"The minimum cost of a traveling salesman tour for 25 cities is {minimum_distance}")




Time spent on loading data is 0.0 second(s)
Time spent on generating subsets is 62.632447719573975 second(s)
Time spent on calculating distance is 0.07407808303833008 second(s)
Time spent on doing dynamic programming is 1727.3957517147064 second(s)

The minimum cost of a traveling salesman tour for 25 cities is 26442.73
